# (Not Working) Calculating Polyspectra from a Simulation in Realtime

Old Method! After defining a system (e.g. a single quantum dot measured by a quantum point contact) we can start a simulation
and watch the polyspectra being calculated in real time. The resulting spectra will be equivalent to the ones calculated 
directly from the stochastic master equation.

In [1]:
import QuantumPolyspectra as qp
from qutip import *
import numpy as np

In [2]:
# ------ Operators and Hamiltonian -----
dot_levels = 2
a = destroy(dot_levels)
n = a.dag() * a
rho_0 = ket2dm(basis(dot_levels, 0)) + ket2dm(basis(dot_levels, 1)) 
rho_0 /= np.trace(rho_0)
H = 0 * n

# ------ System Parameters ------
beta = 5
gamma_out = 0.5
gamma_in = 0.5

c_measure_strength = {
    'a': gamma_out**0.5,
    'a_d': gamma_in**0.5,
}

sc_measure_strength = {
    'n': beta
}

c_ops = {
    'a': a,
    'a_d': a.dag(),
}

sc_ops =  {
    'n': n,
}

e_ops = {
    'n': n,
}

system = qp.System(H, rho_0, c_ops, sc_ops, e_ops, c_measure_strength, sc_measure_strength)



Before starting the simulation we have to define a few parameters for the integration stochastic master equation
and the fourier transform:

In [3]:
f_max = 2 #[kHz]
t_start = 0
t_stop = int(10e1)
n_steps = int(50e3+1)
t = np.linspace(t_start, t_stop, n_steps)
f_nyqist =  n_steps / t_stop / 2

print('f_ny: {:.3f} kHz'.format(f_nyqist))
print('f_min: {:.3f} kHz'.format(2 * f_nyqist / n_steps))
print('')
print('f_max: {} kHz'.format(f_max))
print('number of points: {}'.format(int(f_max // (2 * f_nyqist / n_steps))))




Now we just have to state the operate from which we want to analyze the corresponding measurement:

In [4]:
op = 'n'
title = 'Γ<sub>l</sub> = {:.2f} kHz, Γ<sub>r</sub> = {:.2f} kHz, β<sub>main</sub>= {:.1f} kHz<sup>0.5</sup>'.format(gamma_in, gamma_out, beta)
res = system.numeric_spec(t, op, f_max=f_max, power=1, order=2, max_samples=50, plot_after=25, _solver='milstein', title_in=title, with_noise=True, roll=False)